## Installing packages
* pip install lightgbm
* conda install -c conda-forge xgboost
* pip install xgboost

## **Stack OverFlow: EDA & ML**  

**Elijah Zolduoarrati**  
**Approaches and Techniques:**

* EDA with Pandas and Seaborn
* Find features with strong correlation to target variables questions and answers
* Data preprocessing, converting categorical features mainly (country) to numerical
* apply the basic Regression models of sklearn 
* use gridsearchCV to find the best parameters for each model
* compare the performance of the Regressors and choose best one

**The notebook is organized as follows:**

* **[Part 0: Imports, Settings and switches, Global functions](#Part-0-:-Imports,-Settings,-Functions)**  
* import libraries  
* settings for number of cross validations  
* define functions that are used often

* **[Part 1: Exploratory Data Analysis](#Part-1:-Exploratory-Data-Analysis)**  
1.1 Get an overview of the features (numerical and categorical) and first look on the target variables questions and answers
[shape, info, head and describe](#shape,-info,-head-and-describe)  
[Distribution of the target variable Q](#The-target-variable-:-Distribution-of-questions-and-answers)  
[Numerical and Categorical features](#Numerical-and-Categorical-features)  
[List of features with missing values](#List-of-features-with-missing-values) and Filling missing values using [log transform](#log-transform)  
1.2 Relation of all features to target questions and answers  
[Seaborn regression plots for numerical features](#Plots-of-relation-to-target-for-all-numerical-features)  
[List of numerical features and their correlation coefficient to target](#List-of-numerical-features-and-their-correlation-coefficient-to-target)  
[Seaborn boxplots for categorical features](#Relation-to-questions-and-answers-for-all-categorical-features)  
[List of categorical features and their unique values](#List-of-categorical-features-and-their-unique-values)  
1.3 Determine the columns that show strong correlation to target  
[Correlation matrix 1](#Correlation-matrix-1) : all numerical features determine features with largest correlation to questions and answers

* **[Part 2: Data wrangling](#Part-2:-Data-wrangling)**  
[Dropping all columns with weak correlation to questions and answers](#Dropping-all-columns-with-weak-correlation-to-questions-and-answers)  
[Convert categorical columns to numerical](#Convert-categorical-columns-to-numerical)  
[Checking correlation to SalePrice for the new numerical columns](#Checking-correlation-to-questions-and-answers-for-the-new-numerical-columns)  
use only features with strong correlation to target  
[Correlation Matrix 2 (including converted categorical columns)](#Correlation-Matrix-2-:-All-features-with-strong-correlation-to-questions-and-answers)  
Create datasets for ML algorithms:                                                                          
[OneHotEncoder](#OneHotEncoder)  
[StandardScaler](#StandardScaler)

* **[Part 3: Scikit-learn basic regression models and comparison of results](#Part-3:-Scikit-learn-basic-regression-models-and-comparison-of-results)**  
implement GridsearchCV with RMSE metric for Hyperparameter tuning for these models from sklearn:  
[Linear Regression](#Linear-Regression)  
[Ridge](#Ridge)  
[Lasso](#Lasso)  
[Elastic Net](#Elastic-Net)  
[Stochastic Gradient Descent](#SGDRegressor)  
[DecisionTreeRegressor](#DecisionTreeRegressor)  
[Random Forest Regressor](#RandomForestRegressor)  
[KNN Regressor](#KNN-Regressor)  
Baed on RMSE metric, compare performance of the regressors with their optimized parameters, then explore correlation of the predictions and make submission with mean of best models plot comparison:             
[RMSE of all models](#Comparison-plot:-RMSE-of-all-models)  
[Correlation of model results](#Correlation-of-model-results)  
Mean of best models


Note on scores:  
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed questions and answers. (Taking logs means that errors in predicting questions and answers will affect the result equally.)

# Part 0 : Imports, Settings, Functions

In [1]:
#Visualizing Lib
import seaborn as sns
import matplotlib.pyplot as plt

#Math Lib for some statistics
from scipy import stats
%matplotlib inline
sns.set()

# df preprocessing Lib
import pandas as pd
import numpy as np
pd.set_option('max_columns', 105)

# AI preprocessing lib
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split 

# ML Lib
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
import sklearn.linear_model as linear_model
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor

# warning supressor
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore")
#importing necessary models and libraries

**Settings and switches**

**Here we can choose settings for optimal performance and runtime. For example, nr_cv sets the number of cross validations used in GridsearchCV, and min_val_corr is the minimum value for the correlation coefficient to the target (only features with larger correlation will be used).** 

In [2]:
# setting the number of cross validations used in the Model part 
nr_cv = 5

# switch for using log values for SalePrice and features     
use_logvals = 1    
# target used for correlation 
target_1 = 'questions'
target_2 = 'answers'    
# only columns with correlation above this threshold value  
# are used for the ML Regressors in Part 3
min_val_corr = 0.4    
    
# switch for dropping columns that are similar to others already used and show a high correlation to these     
drop_similar = 1

**Initiate functions:**

In [3]:
def get_best_score(grid):
    
    best_score = np.sqrt(-grid.best_score_)
    print(best_score)    
    print(grid.best_params_)
    print(grid.best_estimator_)
    return best_score

def print_cols_large_corr(df, nr_c, targ) :
    corr = df.corr()
    corr_abs = corr.abs()
    print (corr_abs.nlargest(nr_c, targ)[targ])

def plot_corr_matrix(df, nr_c, targ) :
    
    corr = df.corr()
    corr_abs = corr.abs()
    cols = corr_abs.nlargest(nr_c, targ)[targ].index
    cm = np.corrcoef(df[cols].values.T)

    plt.figure(figsize=(nr_c/1.5, nr_c/1.5))
    sns.set(font_scale=1.25)
    sns.heatmap(cm, linewidths=1.5, annot=True, square=True, 
                fmt='.2f', annot_kws={'size': 10}, 
                yticklabels=cols.values, xticklabels=cols.values
               )
    plt.show()

In [4]:
from subprocess import check_output, call 
print(check_output(["dir", "C:\\Users\\alamo248\\Downloads\\Data\\all_uptt.csv"],shell=True).decode("utf8"))

 Volume in drive C is Windows
 Volume Serial Number is 5843-B50F

 Directory of C:\Users\alamo248\Downloads\Data

10/12/2019  04:07 PM           191,469 all_uptt.csv
               1 File(s)        191,469 bytes
               0 Dir(s)  360,661,278,720 bytes free



In [5]:
#Importing data into dataframe
df =  pd.read_csv('C://Data/all_upt.csv')

In [6]:
#Displaying dataframe
df.head()

,Id,AboutMe,CreationDate,LastAccessDate,DisplayName,Location,country,AboutMe_length,activity_in_months,UpVotes,DownVotes,Reputation,Views,badges,Q_comments,A_comments,P_questions,P_answers
0,7784035,<p>This product may also help you to feel full...,2017-03-29 06:30:11.463 UTC,2017-03-29 06:30:11.463 UTC,berwenally,New York,usa,1407.0,0.0,0,0,1,2,1,10.0,11.0,12.0,13.0
1,9126518,<p>A simple way to keep your hands moisturized...,2017-12-21 10:07:00.237 UTC,2017-12-21 10:07:00.237 UTC,TeresaEWilliams,United States,usa,622.0,0.0,0,0,1,0,1,NaN,NaN,0.0,0.0
2,10021482,<p>Completely covers and protects your body pi...,2018-07-02 12:09:47.843 UTC,2018-07-02 13:30:11.797 UTC,kimberly02,Fremont,usa,370.0,0.0,0,0,1,1,1,NaN,NaN,0.0,0.0
3,2739419,<p>POOL TABLES FOR SALE CONTEMPORARY CLASSIC C...,2013-09-02 10:00:27.57 UTC,2013-09-02 10:00:27.57 UTC,USA Made Pool Tables,California,usa,400.0,0.0,0,0,1,1,1,NaN,NaN,0.0,0.0
4,1440631,<p>I am a Software Engineer</p>\n,2012-06-06 19:21:00.903 UTC,2012-06-06 19:36:13.78 UTC,twinkle,United States,usa,32.0,0.0,0,0,1,0,1,NaN,NaN,0.0,0.0


In [7]:
# subtitute missing values by zero
#df = df.fillna(0)
#Displaying modified dataframe
#df.head()

,Id,AboutMe,CreationDate,LastAccessDate,DisplayName,Location,country,AboutMe_length,activity_in_months,UpVotes,DownVotes,Reputation,Views,badges,Q_comments,A_comments,P_questions,P_answers
0,7784035,<p>This product may also help you to feel full...,2017-03-29 06:30:11.463 UTC,2017-03-29 06:30:11.463 UTC,berwenally,New York,usa,1407.0,0.0,0,0,1,2,1,10.0,11.0,12.0,13.0
1,9126518,<p>A simple way to keep your hands moisturized...,2017-12-21 10:07:00.237 UTC,2017-12-21 10:07:00.237 UTC,TeresaEWilliams,United States,usa,622.0,0.0,0,0,1,0,1,0.0,0.0,0.0,0.0
2,10021482,<p>Completely covers and protects your body pi...,2018-07-02 12:09:47.843 UTC,2018-07-02 13:30:11.797 UTC,kimberly02,Fremont,usa,370.0,0.0,0,0,1,1,1,0.0,0.0,0.0,0.0
3,2739419,<p>POOL TABLES FOR SALE CONTEMPORARY CLASSIC C...,2013-09-02 10:00:27.57 UTC,2013-09-02 10:00:27.57 UTC,USA Made Pool Tables,California,usa,400.0,0.0,0,0,1,1,1,0.0,0.0,0.0,0.0
4,1440631,<p>I am a Software Engineer</p>\n,2012-06-06 19:21:00.903 UTC,2012-06-06 19:36:13.78 UTC,twinkle,United States,usa,32.0,0.0,0,0,1,0,1,0.0,0.0,0.0,0.0


In [8]:
## creating a variable vector
vec = df.iloc[:,[0,6,7,8,9,10,11,12,13,14,15,16,17]]
# Or
# vec = df.loc[:,['Id','DisplayName','Location','country','AboutMe_length','activity_in_months','UpVotes','DownVotes','Reputation','Views','badges','Q_comments','A_comments','P_questions','P_answers']]

In [9]:
vec.head()

,Id,country,AboutMe_length,activity_in_months,UpVotes,DownVotes,Reputation,Views,badges,Q_comments,A_comments,P_questions,P_answers
0,7784035,usa,1407.0,0.0,0,0,1,2,1,10.0,11.0,12.0,13.0
1,9126518,usa,622.0,0.0,0,0,1,0,1,0.0,0.0,0.0,0.0
2,10021482,usa,370.0,0.0,0,0,1,1,1,0.0,0.0,0.0,0.0
3,2739419,usa,400.0,0.0,0,0,1,1,1,0.0,0.0,0.0,0.0
4,1440631,usa,32.0,0.0,0,0,1,0,1,0.0,0.0,0.0,0.0


In [10]:
# Create training and testing sets
vec_train,vec_test= train_test_split(vec, test_size = 0.2, random_state = 0)

# Part 1: Exploratory Data Analysis

## 1.1 Overview of features and relation to target

Let's get a first overview of the train and test dataset
* How many rows and columns are there?  
* What are the names of the features (columns)?  
* Which features are numerical, which are categorical?  
* How many values are missing?  
The **shape** and **info** methods answer these questions. Whereas, the **head** displays some rows of the dataset **describe** gives a summary of the statistics (only for numerical columns)

### shape, info, head and describe

In [15]:
print('-'*100)
print('training sample size')
print(vec_train.shape)
print('-'*100)
print('testing sample size')
print(vec_test.shape)
print('-'*100)
print('training sample features description')
print(vec_train.info())
print('-'*100)
print('testing sample features description')
print(vec_train.info())
print('-'*100)

----------------------------------------------------------------------------------------------------
training sample size
(112147, 13)
----------------------------------------------------------------------------------------------------
testing sample size
(28037, 13)
----------------------------------------------------------------------------------------------------
training sample features description
<class 'pandas.core.frame.DataFrame'>
Int64Index: 112147 entries, 8441 to 43567
Data columns (total 13 columns):
Id                    112147 non-null int64
country               112147 non-null object
AboutMe_length        112147 non-null float64
activity_in_months    112147 non-null float64
UpVotes               112147 non-null int64
DownVotes             112147 non-null int64
Reputation            112147 non-null int64
Views                 112147 non-null int64
badges                112147 non-null int64
Q_comments            112147 non-null float64
A_comments            112147 non-n

* It seems like the trainning and tesing dataframe *(df)* vector *(vec)* consists of 13 columns (12 features excluding Id), as for the training df vec, it has 112147 entries (number of rows). On the other hand,  df test vec has 28037 entries.  
* There are lots of info that is probably related to the dependent variables (target) questions and answers such as badges, reputaion, etc...   
* Maybe other features are not so important for predicting the target, also there might be a strong correlation for some of the features (like activity_in_month).
* For some columns many values are missing: only 7 values for Pool QC in df train and 3 in df test

In [ ]:
# create the independent variable vector
x = df.iloc[:,6:15].values
# Question dependent variable vector 
y = df.iloc[:, -2:-1].values
# Answer dependent variable vector 
z = df.iloc[:,-1:].values

In [ ]:
# create labelEncoder object to transform categorical values into integers
x[:, 0] = LabelEncoder().fit_transform(x[:, 0])
y = LabelEncoder().fit_transform(y)
z = LabelEncoder().fit_transform(z)

In [ ]:
# creating OneHotEncoder object to transform integer categorical values into dummy categorical
x = OneHotEncoder(categorical_features=[0]).fit_transform(x).toarray()

In [ ]:
# Create training and testing sets
x_train,x_test,y_train,y_test,z_train,z_test = train_test_split(x,y,z, test_size = 0.2, random_state = 0)

In [ ]:
# feature scaling
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)